In [1]:
import glob
def read_fl(fnm):
    resp=[]
    f = open(fnm, "r")
    try:
      resp=f.read()
    except Exception as e: print(e)
    return resp

Y_train=glob.glob("book*")
print(Y_train)
x_text=[]

for el in Y_train:
    books=glob.glob(el+"/*.txt")
    for book in books:
        txt=read_fl(book)
        x_text.append(txt)
        
test_files=glob.glob("test_books/*.txt")
test_text=[]
for chapter in test_files:
    print(chapter)
    txt=read_fl(chapter)
    test_text.append(txt)
print(len(test_text), test_text[0],test_text[1])

['books_agatha_christie', 'books_jane_austen', 'books_morrel_david', 'books_arthur_conan_doyle']
test_books/test_dm.txt
test_books/test_ja.txt
2    ser was directly outside. 'Get in,' Teasle said, tugging at his sweaty shirt. 'Damn, for the first of October it sure is hot. I don't know how you tolerate wearing that hot jacket.'
   'I don't sweat.'
   Teasle looked at him. 'Sure you don't.' He dropped his cigarette down a manhole grate by the curb, and they got into the cruiser. Rambo watched the traffic and the people going past. In the bright sun after the dark lunch counter, his eyes hurt. One man walking by the cruiser waved to Teasle, and Teasle waved back, then pulled away from the curb into a break in traffic. He was driving fast this time.
   They went down past a hardware store and a used car lot and old men smoking cigars on benches and women pushing children in strollers.
   'Look at those women,' Teasle said. 'A hot day like this and they don't have the sense to keep their k

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1, 3))
vect.fit(x_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [41]:
print("Размер словаря: {}".format(len(vect.vocabulary_))) 
print("Содержимое словаря:\n {}".format(list(vect.vocabulary_)[10000:10020]))


Размер словаря: 442272
Содержимое словаря:
 ['him inquiringly', 'inquiringly yes', 'yes my', 'friend she', 'had kindly', 'kindly extended', 'extended hospitality', 'hospitality to', 'seven of', 'my countrypeople', 'countrypeople who', 'who alas', 'alas are', 'are refugees', 'refugees from', 'from their', 'their native', 'native land', 'land we', 'we belgians']


In [42]:
bag_of_words = vect.transform(x_text)
print("bag_of_words: {}".format(repr(bag_of_words)))
print("Плотное представление bag_of_words:\n{}".format( bag_of_words.toarray()))

bag_of_words: <4x442272 sparse matrix of type '<class 'numpy.int64'>'
	with 509168 stored elements in Compressed Sparse Row format>
Плотное представление bag_of_words:
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [43]:
X_train = vect.transform(x_text)
print("X_train:\n{}".format(repr(X_train)))

X_train:
<4x442272 sparse matrix of type '<class 'numpy.int64'>'
	with 509168 stored elements in Compressed Sparse Row format>


In [44]:
feature_names = vect.get_feature_names()
print("Количество признаков: {}".format(len(feature_names)))
print("Первые 20 признаков: {}".format(feature_names[:20])) 
print("Признаки с 20010 по 20030: {}".format(feature_names[10010:10030])) 
print("Каждый 2000-й признак:\n{}".format(feature_names[::2000]))

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)

Количество признаков: 442272
Первые 20 признаков: ['000', '000 are', '000 are particularly', '01', '01 it', '01 it seemed', '02', '02 poirot', '02 poirot locked', '03', '03 was', '03 was puzzled', '04', '04 chapter', '04 chapter it', '06', '06 30', '06 30 30', '10', '10 cents']
Признаки с 20010 по 20030: ['all evening though', 'all evie', 'all evie can', 'all except', 'all except shingleton', 'all exclaimed', 'all exclaimed against', 'all expect', 'all expect him', 'all expect jane', 'all expect you', 'all expectation', 'all expectation from', 'all expression', 'all expression john', 'all extraordinary', 'all extraordinary merit', 'all extremely', 'all extremely mysterious', 'all eyes']
Каждый 2000-й признак:
['000', '_you_ cannot', 'account for my', 'advise him', 'again would depress', 'all ended', 'along perhaps', 'amiable qualities must', 'and any', 'and expressed', 'and kindness the', 'and revolver', 'and the wound', 'animal hudson became', 'anybody else however', 'are instituted',

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, Y_train)

In [48]:
for test in range(len(test_files)):
    txt=test_text[test]
    resp=clf.predict_proba(vect.transform([txt]))
    resp2=clf.predict(vect.transform([txt]))
    max_el=max(resp[0])
    idof=list(resp[0]).index(max_el)
    for k in resp[0]:
        print("{0} {1:4.4f} {2:4.4f} {3}".format(test_files[test], k, max_el, resp2, sorted(Y_train)[idof]))
    print("=")
print(sorted(Y_train))

test_books/test_dm.txt 0.0041 0.9505 ['books_morrel_david']
test_books/test_dm.txt 0.0454 0.9505 ['books_morrel_david']
test_books/test_dm.txt 0.0000 0.9505 ['books_morrel_david']
test_books/test_dm.txt 0.9505 0.9505 ['books_morrel_david']
=
test_books/test_ja.txt 0.0000 1.0000 ['books_jane_austen']
test_books/test_ja.txt 0.0000 1.0000 ['books_jane_austen']
test_books/test_ja.txt 1.0000 1.0000 ['books_jane_austen']
test_books/test_ja.txt 0.0000 1.0000 ['books_jane_austen']
=
['books_agatha_christie', 'books_arthur_conan_doyle', 'books_jane_austen', 'books_morrel_david']
